In [ ]:
import clickhouse_connect
import psycopg2
import pandas as pd
import numpy as np

In [ ]:
network = 'base_mainnet'
db_env = 'prod_analytics'

CLICKHOUSE_CONFIG = {
    'host': '',
    'port': '',
    'username': '',
    'database': ''
}

POSTGRES_CONFIG = {
    'host': '',
    'port': '',
    'user': '',
    'password': '',
    'dbname': ''
}

In [ ]:
ch_client = clickhouse_connect.get_client(**CLICKHOUSE_CONFIG)
pg_client = psycopg2.connect(**POSTGRES_CONFIG)

In [ ]:
def get_table_data_ch(client, query):
    return client.query_df(query)

def get_table_data_pg(client, query):
    return pd.read_sql(sql=query, con=client)

In [ ]:
tables_to_compare = [
    {
        'name': 'fct_pool_rewards',
        'cols': ['amount', 'duration']
    },
    {
        'name': 'fct_buyback',
        'cols': ['snx', 'usd', 'snx_price']
    }
]

start_date = pd.to_datetime('2024-01-01')
end_date = pd.to_datetime('2025-01-27')
date_range = [start_date, end_date]

In [ ]:
comparison_results_all = []
for table in tables_to_compare:
    table_name = table['name']

    ch_query = f"""
        select * from synthetix_{table_name}_{network}
        WHERE 
            ts >= toDateTime('{date_range[0].strftime('%Y-%m-%d %H:%M:%S')}') 
            AND ts <= toDateTime('{date_range[1].strftime('%Y-%m-%d %H:%M:%S')}')
    """
    ch_df = get_table_data_ch(ch_client, ch_query)
    ch_df = ch_df.sort_values(by=['ts'], ignore_index=True)

    pg_schema = "prod_base_mainnet"
    pg_query = f"""
        select * from {pg_schema}.{table_name}_{network}
        WHERE 
            ts >= '{date_range[0].strftime('%Y-%m-%d %H:%M:%S')}' 
            AND ts <= '{date_range[1].strftime('%Y-%m-%d %H:%M:%S')}'
    """
    pg_df = get_table_data_pg(pg_client, pg_query)
    pg_df = pg_df.sort_values(by=['ts'], ignore_index=True)

    # Basic comparison metrics
    table_comparison = {
        'table_name': table_name,
        'row_count_match': len(ch_df) == len(pg_df),
        'clickhouse_rows': len(ch_df),
        'postgres_rows': len(pg_df),
        'column_matches': {}
    }
    
    cols_to_compare = table['cols']

    for col in cols_to_compare:
        ch_col = ch_df[col].fillna(0)
        pg_col = pg_df[col].fillna(0)

        diff = ch_col - pg_col
        if ch_col.dtype in ['string', 'object']:
            col_comparison = {
                'exact_match': (ch_col == pg_col).all(),
                'mismatch_count': (~(ch_col == pg_col)).sum()
            }
        else:
            col_comparison = {
                'exact_match': (ch_col == pg_col).all(),
                'mismatch_count': (~(ch_col == pg_col)).sum(),
                'mean_diff': diff.mean(),
                'max_diff': diff.max(),
                'min_diff': diff.min(),
            }
        table_comparison['column_matches'][col] = col_comparison
    comparison_results_all.append(table_comparison)

In [ ]:
for comparison_results in comparison_results_all:
    print(f"\n{'=' * 60}\nComparing Table: {table_name}\n{'=' * 60}")
    # Display the comparison results in a structured format
    print(f"Row Count Match: {comparison_results['table_name']}")
    print(f"Row Count Match: {comparison_results['row_count_match']}")
    print(f"ClickHouse Rows: {comparison_results['clickhouse_rows']}")
    print(f"Postgres Rows: {comparison_results['postgres_rows']}\n")

    print("Column Comparisons:")
    for col, result in comparison_results['column_matches'].items():
        print(f"\n  Column: {col}")
        print(f"    Exact Match: {result['exact_match']}")
        print(f"    Mismatch Count: {result['mismatch_count']}")

        if 'mean_diff' in result:
            print(f"    Mean Difference: {result['mean_diff']:.6e}")
            print(f"    Max Difference: {result['max_diff']:.6e}")
            print(f"    Min Difference: {result['min_diff']:.6e}")

    print("\n" + "-" * 60)